# Geodist Python usage

This notebook demos the public `geodist` Python API. It keeps to the exported surface
available from `import geodist` (no private Rust module access).

Prerequisites when running from the repo root:
- `cd pygeodist && uv sync --all-extras`
- `uv run maturin develop` to build the extension module before importing

Open the notebook from `pygeodist/` (or adjust the sys.path cell below) so imports resolve.


In [9]:
from __future__ import annotations

import pathlib
import sys

# Allow running the notebook from pygeodist/notebooks while using the local source tree.
project_root = pathlib.Path('..').resolve()
python_src = project_root / 'src'
if python_src.is_dir() and str(python_src) not in sys.path:
    sys.path.insert(0, str(python_src))

print(f'Using Python {sys.version.split()[0]}')
print(f'Notebook cwd: {pathlib.Path.cwd()}')
print(f'Import path head: {sys.path[0]}')


Using Python 3.11.11
Notebook cwd: /home/ejones/workspace/eddieland/geodist/pygeodist/notebooks
Import path head: /home/ejones/.local/share/uv/python/cpython-3.11.11-linux-x86_64-gnu/lib/python311.zip


In [10]:
from geodist import (
    EARTH_RADIUS_METERS,
    BoundingBox,
    Ellipsoid,
    GeodesicResult,
    GeodistError,
    InvalidGeometryError,
    Point,
    Point3D,
    geodesic_distance,
    geodesic_distance_on_ellipsoid,
    geodesic_distance_3d,
    geodesic_with_bearings,
    geodesic_with_bearings_on_ellipsoid,
    hausdorff,
    hausdorff_3d,
    hausdorff_directed,
    hausdorff_directed_3d,
    hausdorff_clipped,
    hausdorff_clipped_3d,
    hausdorff_directed_clipped,
    hausdorff_directed_clipped_3d,
)

EARTH_RADIUS_METERS


6371008.8

## Points and tuples

Create latitude/longitude points (degrees) and optional altitude for 3D points.
Each wrapper can round-trip to tuples for interoperability.


In [11]:
nyc = Point(40.7128, -74.0060)
london = Point(51.5074, -0.1278)

print('NYC tuple:', nyc.to_tuple())
print('London tuple:', london.to_tuple())
print('NYC repr:', nyc)

# 3D point with altitude in meters
viewpoint = Point3D(37.7335, -119.5580, 2693.0)
print('3D point:', viewpoint)


NYC tuple: (40.7128, -74.006)
London tuple: (51.5074, -0.1278)
NYC repr: Point(lat=40.7128, lon=-74.006)
3D point: Point3D(lat=37.7335, lon=-119.558, altitude_m=2693.0)


## Great-circle distance and bearings

Use `geodesic_distance` for the spherical fast path (mean-radius approximation) and
`geodesic_with_bearings` for distance plus initial/final bearings.


In [12]:
nyc_to_london_m = geodesic_distance(nyc, london)
bearings: GeodesicResult = geodesic_with_bearings(nyc, london)

print(f'NYC to London: {nyc_to_london_m/1000:.1f} km')
print(f'  initial bearing: {bearings.initial_bearing_deg:.1f} deg')
print(f'  final bearing:   {bearings.final_bearing_deg:.1f} deg')


NYC to London: 5570.2 km
  initial bearing: 51.2 deg
  final bearing:   108.3 deg


## Ellipsoidal geodesics (WGS84 or custom)

Use `geodesic_distance_on_ellipsoid` / `geodesic_with_bearings_on_ellipsoid` for
higher-accuracy results tied to a specific ellipsoid. WGS84 is the default; provide
explicit axes when working with alternate datums or planets.


In [ ]:
wgs84 = Ellipsoid.wgs84()
spherical_nyc_london_m = geodesic_distance(nyc, london)
ellipsoidal_nyc_london_m = geodesic_distance_on_ellipsoid(nyc, london, wgs84)
ellipsoidal_bearings = geodesic_with_bearings_on_ellipsoid(nyc, london, wgs84)

print(f'Spherical mean-radius: {spherical_nyc_london_m/1000:.2f} km')
print(f'WGS84 ellipsoidal:     {ellipsoidal_nyc_london_m/1000:.2f} km')
print(
    f'Bearings (init/final): '
    f'{ellipsoidal_bearings.initial_bearing_deg:.2f} deg / '
    f'{ellipsoidal_bearings.final_bearing_deg:.2f} deg'
)

mars_semi_major_m = 3_396_190.0
mars_flattening = 1.0 / 169.8944472
mars_semi_minor_m = mars_semi_major_m * (1.0 - mars_flattening)
mars = Ellipsoid(mars_semi_major_m, mars_semi_minor_m)
pathfinder = Point(19.26, 326.75)
curiosity = Point(-4.765700445, 137.39820983)
mars_distance_m = geodesic_distance_on_ellipsoid(curiosity, pathfinder, mars)
print(f'Mars pathfinder->curiosity distance: {mars_distance_m/1000:.2f} km')


## 3D straight-line distance (ECEF chord)

`geodesic_distance_3d` computes a straight-line chord through space using
latitude/longitude in degrees and altitude in meters.


In [13]:
# Yosemite Valley viewpoint to Yosemite Falls overlook (approximate)
valley_floor = Point3D(37.7331, -119.5586, 1200.0)
clifftop = Point3D(37.7335, -119.5580, 2693.0)

line_of_sight_m = geodesic_distance_3d(valley_floor, clifftop)
print(f'Line-of-sight distance: {line_of_sight_m:.1f} m')


Line-of-sight distance: 1494.6 m


## Hausdorff distance over point sets

Hausdorff compares two sets of points; directed answers
"how far is set A from being covered by B?" while symmetric checks the max distance either direction.


In [14]:
trail_a = [
    Point(47.6205, -122.3493),
    Point(47.6220, -122.3470),
    Point(47.6235, -122.3440),
]
trail_b = [
    Point(47.6200, -122.3490),
    Point(47.6215, -122.3465),
    Point(47.6230, -122.3435),
]

symmetric_m = hausdorff(trail_a, trail_b)
directed_ab_m = hausdorff_directed(trail_a, trail_b)
directed_ba_m = hausdorff_directed(trail_b, trail_a)

print(f'Symmetric Hausdorff: {symmetric_m:.2f} m')
print(f'Directed A->B:       {directed_ab_m:.2f} m')
print(f'Directed B->A:       {directed_ba_m:.2f} m')


Symmetric Hausdorff: 67.05 m
Directed A->B:       67.05 m
Directed B->A:       67.05 m


## Bounding-box clipping to ignore outliers

Clipped variants restrict evaluation to a latitude/longitude envelope,
which is useful when one set contains distant outliers you want to ignore.


In [15]:
outlier = Point(10.0, 10.0)
trail_b_with_outlier = trail_b + [outlier]

bbox = BoundingBox(47.61, 47.64, -122.36, -122.33)

raw_hausdorff_m = hausdorff(trail_a, trail_b_with_outlier)
clipped_hausdorff_m = hausdorff_clipped(trail_a, trail_b_with_outlier, bbox)

print(f'Hausdorff with outlier: {raw_hausdorff_m/1000:.2f} km')
print(f'Clipped Hausdorff:      {clipped_hausdorff_m:.2f} m')


Hausdorff with outlier: 12074.82 km
Clipped Hausdorff:      67.05 m


## 3D Hausdorff distances for altitude-aware tracks

When trail segments vary in altitude, the 3D Hausdorff helpers compare ECEF
coordinates so vertical separation counts toward the distance. Use the directed
variants to see how far each path deviates from the other and the symmetric
result to summarize both directions.


In [ ]:
ridge_a = [
    Point3D(37.7331, -119.5586, 1200.0),
    Point3D(37.7340, -119.5570, 1500.0),
    Point3D(37.7350, -119.5550, 2100.0),
]
ridge_b = [
    Point3D(37.7332, -119.5584, 1195.0),
    Point3D(37.7342, -119.5568, 1510.0),
    Point3D(37.7348, -119.5552, 2080.0),
]

ridge_symmetric_m = hausdorff_3d(ridge_a, ridge_b)
ridge_directed_ab_m = hausdorff_directed_3d(ridge_a, ridge_b)
ridge_directed_ba_m = hausdorff_directed_3d(ridge_b, ridge_a)

print(f'3D symmetric Hausdorff: {ridge_symmetric_m:.2f} m')
print(f'3D directed A->B:       {ridge_directed_ab_m:.2f} m')
print(f'3D directed B->A:       {ridge_directed_ba_m:.2f} m')


### Clipping 3D paths to ignore unrelated detours

The clipping envelope applies to the horizontal latitude/longitude bounds while
still accounting for altitude in the distance calculation. This helps isolate
sections of a trail without being skewed by faraway diversions.


In [ ]:
ridge_outlier = ridge_b + [Point3D(37.8000, -119.7000, 500.0)]
ridge_bbox = BoundingBox(37.73, 37.74, -119.56, -119.55)

ridge_raw_m = hausdorff_3d(ridge_a, ridge_outlier)
ridge_clipped_m = hausdorff_clipped_3d(ridge_a, ridge_outlier, ridge_bbox)

print(f'3D Hausdorff with detour: {ridge_raw_m/1000:.2f} km')
print(f'3D Hausdorff clipped:     {ridge_clipped_m:.2f} m')


## Error handling and validation

Geometry inputs are validated before computation. Invalid coordinates raise
`InvalidGeometryError`, while empty inputs surface as `GeodistError`. Catch
these exceptions to provide clearer user-facing messages in data pipelines.


In [ ]:
try:
    Point(95.0, 0.0)
except InvalidGeometryError as exc:
    print(f'Invalid geometry: {exc}')

try:
    hausdorff([], trail_b)
except GeodistError as exc:
    print(f'Empty input error: {exc}')
